In [13]:
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(ggthemes)
library(zoo)


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [8]:
lists <- c("BM-1","BM-21","BM-24","BM-4","BM-46","BM-5","BM-6")

In [50]:
annotation <- read.delim("/gpfs/gsfs12/users/wangy80/TK105/data/00ref/dmel-all-r6.46.gene.anno.txt",header=F,sep="\t")
names(annotation) <- c("chr","start","end","strand","GeneID","Gene_name")
annotation <- annotation[which(annotation$chr=="2L" | annotation$chr=="2R" | annotation$chr=="3L" | annotation$chr=="3R"  | annotation$chr=="X"),]

In [69]:
for (i in 1:length(lists)) {
    
    DNA_res <- read.delim(paste0("/gpfs/gsfs12/users/wangy80/TK117/results/03map_reads2/filter/",lists[i],".w100000_s10000.txt"),sep="\t",header=F)
    names(DNA_res) <- c("chr","start","end","count")
    DNA_res2 <- DNA_res[which(DNA_res$chr == "2L" | DNA_res$chr == "2R" | DNA_res$chr == "3L" | DNA_res$chr == "3R" |  DNA_res$chr == "X"), ]
    DNA_res2$class <- "DNA"
    
    name <- gsub("-","",lists[i])
    RNA_res_Adult <- read.delim(paste0("/gpfs/gsfs12/users/wangy80/TK105/Age/DE/","Adult_Condition_",name,"_vs_Control_Adult_DESeq2.txt"),sep="\t",header=T)
    RNA_res_Adult$class <- "Adult"
    RNA_res_Adult$log2FoldChange <- ifelse(RNA_res_Adult$padj<0.05,RNA_res_Adult$log2FoldChange,0)
    RNA_res_Adult2 <- inner_join(RNA_res_Adult,annotation,by="Gene_name")
    
    RNA_res_L3 <- read.delim(paste0("/gpfs/gsfs12/users/wangy80/TK105/Age/DE/","L3_Condition_",name,"_vs_Control_L3_DESeq2.txt"),sep="\t",header=T)
    RNA_res_L3$class <- "L3"
    RNA_res_L3$log2FoldChange <- ifelse(RNA_res_L3$padj<0.05,RNA_res_L3$log2FoldChange,0)
    RNA_res_L3.2 <- inner_join(RNA_res_L3,annotation,by="Gene_name")
    
    # Define window size
    window_size = 50
    
    # Sort genes by location
    sorted_genes.Adult <- RNA_res_Adult2 %>% arrange(chr, start)
        
    # Calculate sliding window averages for each chromosome
    sliding_window_averages.Adult <- sorted_genes.Adult %>%
    group_by(chr) %>%
    mutate(window_average = rollapply(log2FoldChange, window_size, mean, align = "left", fill = NA)) %>%
    filter(!is.na(window_average))
    
    # Sort genes by location
    sorted_genes.L3 <- RNA_res_L3.2 %>% arrange(chr, start)
        
    # Calculate sliding window averages for each chromosome
    sliding_window_averages.L3 <- sorted_genes.L3 %>%
    group_by(chr) %>%
    mutate(window_average = rollapply(log2FoldChange, window_size, mean, align = "left", fill = NA)) %>%
    filter(!is.na(window_average))
        
    RNA_res <- rbind(sliding_window_averages.Adult,sliding_window_averages.L3)
    RNA_res2 <- RNA_res[,c(8,9,10,13,7)]
    names(RNA_res2) <- c("chr","start","end","count","class")
   
    res <- rbind(DNA_res2,RNA_res2)
    
    res$class <- factor(res$class,levels=c("Adult","L3","DNA"))
    #Plot the sliding window averages
    p <- ggplot(res, aes(x = start, y = count,color=class)) +
          #facet_wrap(~chr,ncol=1) +
          facet_grid(class ~ chr,scales="free") +
          geom_line(alpha=1) +
          labs(x = "Location", y = "Scale", title = "Sliding Window by Chromosome") +
          scale_color_brewer(palette="Paired") + 
          #ylim(c(-0.5,0.5)) +
          geom_hline(yintercept=0,color="#999999",linetype=2) +
          theme_few()
    
    ggsave(filename = paste0(lists[i],".pdf"), plot = p, path = "./Plots", width = 30, height = 5)
}